In [ ]:
# Install required packages (uncomment if needed)
# !pip install transformers peft datasets accelerate bitsandbytes torch


In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import os


## Configuration


In [ ]:
# Model configuration
MODEL_NAME = "microsoft/DialoGPT-small"  # Change to your preferred model
OUTPUT_DIR = "./lora_output"
MAX_LENGTH = 512

# LoRA configuration
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.1
TARGET_MODULES = ["c_attn", "c_proj"]  # Adjust based on your model architecture

# Training configuration
BATCH_SIZE = 4
LEARNING_RATE = 2e-4
NUM_EPOCHS = 3
WARMUP_STEPS = 100
SAVE_STEPS = 500
LOGGING_STEPS = 100


## Load Model and Tokenizer


In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"Model loaded: {MODEL_NAME}")
print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M parameters")


KeyboardInterrupt: 

## Setup LoRA


In [ ]:
# Configure LoRA
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()


## Prepare Dataset


In [ ]:
# Example dataset - replace with your own data
# Format: list of text strings
train_texts = [
    "This is an example training text.",
    "Another example for fine-tuning.",
    "Add your training data here.",
    # Add more examples...
]

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length"
    )

# Create dataset
train_dataset = Dataset.from_dict({"text": train_texts})
train_dataset = train_dataset.map(tokenize_function, batched=True)

print(f"Dataset size: {len(train_dataset)}")


## Training Setup


In [ ]:
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to True for masked language modeling
)

# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=3,
    fp16=True,
    gradient_accumulation_steps=1,
    report_to="none",
    remove_unused_columns=False,
)


## Train Model


In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

# Start training
print("Starting training...")
trainer.train()

print("Training completed!")


## Save Model


In [ ]:
# Save the LoRA adapter
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Model saved to {OUTPUT_DIR}")


## Load and Use Trained Model


In [ ]:
# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
from peft import PeftModel
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

# Merge LoRA weights (optional)
# model = model.merge_and_unload()

# Example inference
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test generation
test_prompt = "Hello, how are you?"
print(f"Prompt: {test_prompt}")
print(f"Generated: {generate_text(test_prompt)}")
